In [ ]:
%matplotlib notebook
import GWDistancePipeline
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import h5py
import matplotlib.mlab as mlab
import scipy
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt
import mpmath
from scipy.optimize import curve_fit
# import readligo as rl

# Origional Strain Model: 

$$d_{new}=nd_{old} = n \left[\frac{2c}{h_+} \left( \frac{G \mathcal{M}}{c^3}\right)^{5/3} \Omega^{\frac{2}{3}}(t) \left(1+cos^2i\right) cos2\Phi(t)\right]$$ 

$$n = \frac{d_L}{d_0} = \frac{d_{old}}{d_{actual}}$$

$$\mathcal{A_{+}}=h_{+}= \frac{d_0}{d_L}\frac{1+cos^2i}{2} \rightarrow \left(1+cos^2i\right) = \frac{2d_Lh_{+}}{d_0}$$

$$h_+ = \frac{1+cos^2i}{2n}$$

# Updated Strain Model:

(3.2) $$h_{IFO} = A h_+ + B h_\times$$

(3.4a) $$h_+ \equiv \frac{2 \mathcal{M}}{d} \left(\pi \mathcal{M} \mathcal{f} \right)^{2/3} \left(1+cos^2i\right) cos\left(\Phi + \Psi \right) = \frac{2 \mathcal{M}}{d} \left(\pi \mathcal{M} \mathcal{f} \right)^{2/3} \left(1+cos^2i\right) cos\left(2\Phi(t) \right)$$

(3.4b) $$h_\times \equiv \frac{4 \mathcal{M}}{d}\left(\pi \mathcal{M} \mathcal{f} \right)^{2/3} \left(cosi\right) sin\left(\Phi + \Psi \right) = \frac{4 \mathcal{M}}{d}\left(\pi \mathcal{M} \mathcal{f} \right)^{2/3} \left(cosi\right) sin\left(2\Phi(t)\right)$$ 

(3.5d) $$\mathcal{M} \equiv \left(1+z\right) \mu^{3/5} M^{2/5}$$   

(3.5e) $$\Phi(t) \equiv -2 \left(\frac{T-t}{5 \mathcal{M}}\right)^{5/8}$$ 

(3.5f) $$\mathcal{f} \equiv \frac{1}{\pi \mathcal{M}} \left[\frac{5}{256} \frac{\mathcal{M}}{T-t}\right]^{3/8}$$


### Variable Explanation: 

$\mu \equiv \frac{m_1 m_2}{m_1 + m_2}$

$T = $ Newtonian "moment of coalescence"

$\Psi(t=T) = \Phi(T) $ phase of binary system when $t=T$

$M =$ total combined mass of system 

$\mu = \frac{m_1m_2}{m_1+m_2} = $ reduced mass

$\mathcal{f} =$ gravitational radiation frequency

# Expressing updated model in terms that can be plotted
1. Express $h_{IFO}$ in terms of $\frac{h_+}{h_\times}$ $\rightarrow$ $h_{IFO} = h_\times \left(A \left(\frac{h_+}{h_\times}\right)+B\right)$

2. Find the ratio $\frac{h_+}{h_\times} = \frac{1}{2}cot\left(2 \Phi\right)\left(sec(i) + cos(i)\right)$

3. Plug $\frac{h_+}{h_\times}$ and $h_\times$ into $h_{IFO}$ to get the final expression...

$$\boxed{h_{IFO} = 2 K sin (2\Phi) \left[\frac{A}{2}cot(2\Phi) \left(1+cos^2i\right) + B cosi\right]}$$

where $K \equiv \frac{2 \mathcal{M}}{d} \left(\pi \mathcal{M}\mathcal{f}\right)^{2/3}$

**source: Fin et al. 2008, https://arxiv.org/pdf/gr-qc/9301003.pdf

# Helper Functions

In [ ]:
# FFT function
def FTA(strain, time, sampling_freq, plot=False):
    
    fft = np.fft.rfft(strain) / sampling_freq
    freq = np.fft.rfftfreq(len(strain)) * sampling_freq
    
    if plot == True: 
        plt.figure(figsize = (8,5))
        plt.loglog(freq, abs(fft))
        plt.xlim(10,2000)
        plt.grid()
        plt.xlabel('Freq (Hz)')
        plt.ylabel('Strain / Hz')
        plt.show()
    return freq, fft

# Power Spectral Density (PSD) function
def PSD(strain, fs, window, plot=False):
    NFFT = 4 * fs
    NOVL = 1 * NFFT / 2
    pxx, freqs = mlab.psd(strain, Fs=fs, NFFT=NFFT, window=window, noverlap=NOVL)
    if plot == True: 
        plt.figure(figsize = (8,5))
        plt.loglog(freqs, pxx)
        plt.xlim(10,2000)
        plt.grid()
        plt.ylabel('PSD')
        plt.xlabel('Freq (Hz)')
        plt.show()
    return pxx, freqs

# Amplitude Spectral Density (ASD) function
def ASD(strain, sampling_freq, plot=False):
    pxx, freqs = mlab.psd(strain, Fs=sampling_freq, NFFT=2*sampling_freq)
    if plot == True: 
        plt.figure(figsize = (8,5))
        plt.loglog(freqs, np.sqrt(pxx))
        plt.xlim(10,2000)
        plt.grid()
        plt.ylabel('Strain / Hz$^{1/2}$')
        plt.xlabel('Freq (Hz)')
        plt.show()
    return np.sqrt(pxx), freqs

# Getting Segments of Data
* GWOSC Coding Tutorial: https://www.gw-openscience.org/tutorial02/

In [ ]:
GW150914_file = 'H-H1_LOSC_4_V2-1126257414-4096.hdf5' # H1 strain
GW150914d = h5py.File(GW150914_file, 'r') #data file

In [ ]:
def GetSegments(data, plot_raw=False): 
    
    N = 10000
    sf = 4096 # sampling freq in Hz
    dt = N / sf
    
    # load in the data and chose the good reasons
    strain = data['strain']['Strain'].value
    ts = data['strain']['Strain'].attrs['Xspacing']
    
    metaKeys = data['meta'].keys()
    meta = data['meta']
    
    gpsStart = meta[('GPSstart')].value
    duration = meta[('Duration')].value
    
    dqInfo = data['quality']['simple']
    bitnameList = dqInfo['DQShortnames'].value
    nbits = len(bitnameList)
    qmask = dqInfo['DQmask'].value
    
    gpsEnd = gpsStart + len(qmask)
    time = np.arange(gpsStart, gpsEnd, ts)
    t_center = 1126259462 # approximate time of merger event for GW150914
    
    print('Current Version: V.1 (7/18/2021)') # Don't forget to update as necessary
    print('Hard coded for GW150914 to determine segments of good data in the "BURST" category.')
    
    # Chose the correct type of data 
    sci = (qmask >> 0) & 1  # sci = 1 when data passes the "DATA" category tests; sci = 0 when it fails
    burst1  = (qmask >> 4) & 1 # sets locations where qmask >> 4 == 1
    burst2  = (qmask >> 5) & 1 
    burst3  = (qmask >> 6) & 1 
    
    # Create good_data segments
    dummy1 = np.zeros(burst1.shape)
    masked_dummy1 = np.ma.masked_array(dummy1, np.logical_not(burst1))
    
    dummy2 = np.zeros(burst2.shape)
    masked_dummy2 = np.ma.masked_array(dummy2, np.logical_not(burst2))
    
    dummy3 = np.zeros(burst3.shape)
    masked_dummy3 = np.ma.masked_array(dummy3, np.logical_not(burst3)) 
    
    # locate regions of good data inside masked_dummy array
    segments1 = np.ma.flatnotmasked_contiguous(masked_dummy1)
    segments2 = np.ma.flatnotmasked_contiguous(masked_dummy2)
    segments3 = np.ma.flatnotmasked_contiguous(masked_dummy3)

    # Create list of segements seperated by time_ax
    segList1 = [(int(seg.start+gpsStart), int(seg.stop+gpsStart)) for seg in segments1]
    segList2 = [(int(seg.start+gpsStart), int(seg.stop+gpsStart)) for seg in segments2]
    segList3 = [(int(seg.start+gpsStart), int(seg.stop+gpsStart)) for seg in segments3]
    
    # Plot Raw Data 
    if plot_raw == True: 
        plt.figure(figsize = (9,5))
        plt.plot(time - time[0], strain)
        plt.xlabel('GPS time [s]')
        plt.ylabel('Strain')
        plt.grid()
        plt.show()

    return strain, time, dt, ts, sf, segList1, segList2, segList3

In [ ]:
strain, time, dt, ts, sf, segList1, segList2, segList3 = GetSegments(GW150914d, plot_raw=True)

# Chosing good data

In [ ]:
# Function to get good regions of data 
def GoodData(time_ax, strain_ax, dt, segList, plot=False, plt_bin=False):
    # Returns & plots regions of good quality data for a given data file and good_data segments
    
    print('There are ' + str(len(segList)) + ' regions of good data in the segment list.')
    print('')
    print('Which segment would you like to choose?')
    seg = input('Segment: ')
    
    t_start = segList[int(seg)][0]
    t_stop = segList[int(seg)][1]
    print('')
    print('--------------')
    print('')
    print('Chosen segment: ' + str(seg))
    print('GPS Start time: ' + str(t_start) + 's')
    print('GPS Stop time: ' + str(t_stop) + 's')

    
    ind_start = np.where(np.round(time_ax) == t_start)[0][0]
    ind_stop = np.where(np.round(time_ax) == t_stop)[0][0]
    
    #if len(segList) == 1: 
    #    strain_seg = strain_ax[ind_start[0][0]:]
    #    time_seg = time_ax[ind_start[0][0]:]
    
    strain_seg = strain_ax[ind_start:ind_stop]
    time_seg = time_ax[ind_start:ind_stop]
    
    # determine the time since GPS start (in seconds)
    real_time = np.linspace(0, abs(time_seg[0] - time_seg[-1]), num = len(strain_seg)) #currently not worrking 7/20/21
    

    if plot == True:
        plt.figure(figsize=(8,5))
        plt.plot(time_seg, strain_seg)
        plt.title('Segment: ' + str(seg))
        plt.xlabel('GPS Time' )
        plt.ylabel('Strain')
        #plt.title('GW150914')  might want to add in a name file eventually 
        plt.grid()
        plt.show()
        
    '''if plt_bin == True:
        N = 10000 # bin size of data to help with visualization
        plt.figure(figsize=(8,5))
        plt.plot(time_seg, strain_seg)
        
        plt.title('Binned Segment:' + str(seg))
        #plt.title('Binned Segment[0:' + str(N) + ']: '+ str(seg))
        plt.xlabel('Seconds since GPS ' + str(time_seg[0]) )
        plt.ylabel('Strain')
        #plt.title('GW150914')  might want to add in a name file eventually 
        plt.grid()
        plt.show()'''
        
    #return ind_start, ind_stop , strain_seg  
    return strain_seg,time_seg, real_time

In [ ]:
strain_seg,time_seg, real_time = GoodData(time, strain, dt, segList1, plot = True, plt_bin = False)

# Tukey Window

In [ ]:
def Window(strain, time_ax, fs, plot_window=False, plot_raw=False, oplot=False):
    
    time_center = 1126259462 # For GW150914
    window_len = 4 * fs
    dwindow = scipy.signal.tukey(window_len, alpha = 1. / 4)
    
    indxt = np.where((time_ax >= time_center - 2) & (time_ax < time_center + 2))
    
    window_strain = strain[indxt] * dwindow
    window_time = time_ax[indxt]
    
    
    if plot_raw == True:
        plt.figure(figsize=(8,5))
        plt.plot(time_ax[indxt]-time_center, strain[indxt], label = 'Raw')
        plt.xlabel('GPS time')
        plt.ylabel('Strain')
        plt.legend()
        plt.grid()
        plt.show()
        
    if plot_window == True:
        plt.figure(figsize=(8,5))
        plt.plot(window_time-time_center, window_strain, label = 'Windowed')
        plt.xlabel('GPS time')
        plt.ylabel('Strain')
        plt.legend()
        plt.grid()
        plt.show()
        
    if oplot == True: 
        plt.figure(figsize=(8,5))
        plt.plot(time_ax[indxt]-time_center, strain[indxt], label = 'Raw')
        plt.plot(window_time-time_center, window_strain, label = 'Windowed')
        plt.xlabel('Time [s]')
        plt.ylabel('Strain')
        plt.legend()
        plt.grid()
        plt.show()
        
    return window_strain, window_time, dwindow, time_center
    # source: https://github.com/losc-tutorial/Data_Guide/blob/master/Guide_Notebook.ipynb

In [ ]:
window_strain, window_time, window, time_center = Window(strain_seg, time_seg, sf, plot_window=False, plot_raw=False, oplot=True)

# Whiten Spectrum

In [ ]:
def WhiteWindow(strain, time_ax, fs, dt, phase_shift=0, time_shift=0
                , plot_window=False, plot_raw=False, oplot=True, psd=False, white_window=False):
    '''
    Places a Tukey window around the raw strain data and 
        whitens strain data given a raw PSD and sampling frequency, can also apply phase and time shift to data 
    
    Args: 
        strain(ndarray): strain data 
        interp_psd(interpolating fuinction): function that takes in freq data and outputs 
            the avg pwr at the freq
        dt(float): sample time interval for data
        phase_shift(float, optional): phase shift to apply to the whitened data 
        time_shift(float, optional): time shift to apply to whitened data (s)
    '''
    
    # Window
    window_strain, window_time, window, time_center = Window(strain, time_ax, fs
                                                             , plot_window=plot_window, plot_raw=plot_raw, oplot=oplot)
    
    # FFT
    Nt = len(window_strain)
    freqs = np.fft.rfftfreq(Nt, dt)
    hf = np.fft.rfft(window_strain)
    
    # apply phase to strain data
    hf = hf * np.exp(np.exp(-1.j * 2 * np.pi * time_shift * freqs - 1.j * phase_shift))
    
    # normalization & whiten data 
    pxx, f_psd = PSD(strain, fs, window, plot=psd)  
    
    norm = 1. / np.sqrt(1./(dt*2)) # normalization factor 
    white_hf = hf / np.sqrt(pxx) * norm # whitening: transform to freq domain, divide by square root of psd 
    white_ht = np.fft.irfft(white_hf, n = Nt) # ifft the whitened spectrum back into strain data
    
    if white_window == True: 
        plt.figure(figsize=(10,4))
        plt.plot(window_time-time_center, white_ht, label = 'Whitened')
        plt.xlabel('Time [s]')
        plt.ylabel('Strain')
        plt.legend()
        plt.grid()
        plt.show()
    
    return white_ht

In [ ]:
white_ht = WhiteWindow(strain_seg, time_seg, sf, dt, phase_shift=0, time_shift=0
                       , plot_window=False, plot_raw=False, oplot=True, psd=True, white_window=True)


# Bandpass Filter

In [ ]:
def bandpass(strain, fband, fs,time, t_event, bp_full=False, event=False):
    bb, ab = butter(4, [fband[0]*2./fs , fband[1]*2./fs], btype='band')
    normalization = np.sqrt((fband[1]-fband[0])/(fs/2))
    strain_bp = filtfilt(bb, ab, strain)/normalization
    
    # Find the chirp signal
    chirp_ind = np.where(strain_bp == max(strain_bp))[0][0]
    chirp_strain = strain_bp[chirp_ind-500:chirp_ind+200]
    chirp_time = time[chirp_ind-500:chirp_ind+200] - t_event
    
    if bp_full == True:
        # plot full whitened and bandpassed data
        plt.figure(figsize=(10,4))
        plt.plot(window_time-time_center, strain_bp/2000, label = 'Whitened & Bandpassed') # might need to define windowtime in here 
        plt.title('GW150914 Full Event')
        plt.xlabel('Time [s]')
        plt.ylabel('Strain $(10^{-21})$')
        plt.legend()
        plt.grid()
        plt.show()
        
    if event == True:
        # plot full whitened and bandpassed data
        plt.figure(figsize=(10,4))
        plt.plot(time[chirp_ind-500:chirp_ind+200] - t_event, strain_bp[chirp_ind-500:chirp_ind+200]/2000
                 , label = 'Whitened & Bandpassed')
        plt.title('GW150914 Chirp Signal')
        plt.xlabel('Time [s]')
        plt.ylabel('Strain $(10^{-21})$')
        plt.legend()
        plt.grid()
        plt.show()
        
    return strain_bp, chirp_strain, chirp_time

In [ ]:
strain_bp, chirp_strain, chirp_time = bandpass(white_ht, [70., 400.], sf, window_time, time_center
                                               , bp_full=True, event=True)

In [ ]:
len(chirp_time), len(chirp_strain), len(window)

# Model Fitting 

### Necessary Experessions

$$\boxed{h_{IFO} = 2 K sin (2\Phi) \left[\frac{A}{2}cot(2\Phi) \left(1+cos^2i\right) + B cosi\right]}$$

$$K\equiv \frac{2 \mathcal{M}}{d} \left(\pi \mathcal{M}\mathcal{f}\right)^{2/3}$$

$$\Phi(t) \equiv -2 \left(\frac{T-t}{5 \mathcal{M}}\right)^{5/8}$$ 
    
$$\mathcal{f} \equiv \frac{1}{\pi \mathcal{M}} \left[\frac{5}{256} \frac{\mathcal{M}}{T-t}\right]^{3/8}$$


In [ ]:
def h_IFO(K, phi, A, B, i):
    return 2 * K * np.sin(2*phi) *( (A/2) * (np.cos(2*phi)/np.sin(2*phi)) * (1+np.cos(i)**2) + B * np.cos(i))

In [ ]:
def K(M_c, d, f):
    return ((2*M_c)/d) * (np.pi * M_c * f) **(2/3)

In [ ]:
def Phi(chirp_mass, T, t):
    return abs(-2 * ((T-t) / (5* chirp_mass))**(5/8))  # since its being passed into cos, negative shouldnt matter
    # check to see if T-t makes this return an array 

In [ ]:
def F(chirp_mass, T, t):
    return (1/(np.pi * chirp_mass)) * ( (5/256) * (chirp_mass/(T-t)))**(3/8) 
    # check to see if T-t makes this return an array 

In [ ]:
# Data for GW150914

M_c = 28.6 # from LIGO GWTC-1
t = window_time - t_merge_centered
t_merge_gps = 1126259462 #GPS time
t_merge_centered = window_time[np.where(strain_bp == max(strain_bp))[0][0]]-t_merge_gps
d = 440 # Mpc
len(t), len(strain_bp)

In [ ]:
np.where(strain_bp == max(strain_bp))[0][0]

In [ ]:
phi = Phi(M_c, t_merge_centered, t) 
phi[np.where(np.isnan(phi) == True)[0]] = 0
np.where(phi == 0)

In [ ]:
f = F(M_c, t_merge_centered, t)
f[np.where(np.isnan(f) == True)[0]] = 0
np.where(f == 0)

In [ ]:
k = K(M_c, d, f)
k[np.where(np.isnan(k) == True)[0]] = 0
len(k)

In [ ]:
p, cove = curve_fit(h_IFO, t, strain_bp, p0=[]) 
print('actual inc: i ~ 1.225 rad')
print('inc est: ' + str(np.round(p[1], 4)) + ' rad')
p

# Spectrogram

Y'all don't need to worry about this part...unless you're interested lol
(source: http://www.ceri.memphis.edu/people/mwithers/CERI7106/other/GW150914_tutorial.html)

In [ ]:
NFFT = len(window)
NOVL = NFFT*15/16
winodow = np.blackman(NFFT)
delta = 10.
len(window), len(window_strain), NFFT

In [ ]:
plt.figure()
spec_H1, freqs_spec, bins, im = plt.specgram(strain_bp, NFFT=NFFT, Fs=sf, window=window, 
                                        noverlap=NOVL, cmap='viridis', xextent = ([-delta, delta]))
plt.colorbar()
plt.axis([-0.5, 0.5, 0, 500])
plt.ylabel('Freq [Hz]')
plt.xlabel('Time [s]')
plt.show()

In [ ]:
len(window), len(window_strain)